# AI Agent using LangChain

In [1]:
import requests
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from enum import Enum

load_dotenv()

True

In [2]:
from langchain.tools import tool

# Defining Tools

In [13]:

class SearchProductInput(BaseModel):
    product_query: str = Field(..., description="The query string for the product search")


@tool(args_schema=SearchProductInput)
def search_product(product_query):
    """Search for products on Amazon based on a query."""
    
    url = "https://real-time-amazon-data.p.rapidapi.com/search"

    querystring = {"query":product_query,
                   "page":"1",
                   "country":"US",
                   "sort_by":"RELEVANCE",
                   "product_condition":"ALL",
                   "is_prime":"false",
                   "deals_and_discounts":"NONE"}
    
    headers = {
    	"x-rapidapi-key": "<your rapidapi api key>",
    	"x-rapidapi-host": "real-time-amazon-data.p.rapidapi.com"
    }
    
    response = requests.get(url, headers=headers, params=querystring)
    
    return response.json()["data"]["products"][:3]


class AddToCartInput(BaseModel):
    product_id: str = Field(..., description="Unique identifier for the product to be added to the cart.")
    quantity: int = Field(..., description="The quantity of the product to add to the cart.")
    
cart = []
@tool(args_schema=AddToCartInput)
def add_to_cart(product_id, quantity):
    """Adds a product to the shopping cart with the specified quantity."""
    global cart
    cart.append({"product_id": product_id, "quantity": quantity})
    return True


@tool
def summarize_cart():
    """Returns the current contents of the cart"""
    global cart
    return cart


class PaymentMethod(str, Enum):
    COD = "COD"
    CreditCard = "CreditCard"
    DebitCard = "DebitCard"
    UPI = "UPI"
    NetBanking = "NetBanking"

class BuyProductInput(BaseModel):
    payment_method: PaymentMethod = Field(..., description="The method of payment to be used for the purchase.")

@tool(args_schema=BuyProductInput)
def buy_product(payment_method):
    """Processes the purchase of a product using the specified payment method."""
    if payment_method == "COD":
        response = "Order confirmation sent to registered email"
    else:
        response = generate_payment_link(cart)
    return response

# Util function - not a tool to be provided to our agent
def generate_payment_link(cart):
    # TODO: actually implement this function
    return "sample.link.com"

In [9]:
search_product.name

'search_product'

In [10]:
search_product.description

'Search for products on Amazon based on a query.'

In [11]:
from langchain_core.utils.function_calling import convert_to_openai_function

In [12]:
convert_to_openai_function(search_product)

{'name': 'search_product',
 'description': 'Search for products on Amazon based on a query.',
 'parameters': {'properties': {'product_query': {'description': 'The query string for the product search',
    'type': 'string'}},
  'required': ['product_query'],
  'type': 'object'}}

In [14]:
tools = [search_product, add_to_cart, summarize_cart, buy_product]

# Function Calling through Langchain

In [16]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [23]:
functions = [convert_to_openai_function(f) for f in tools]

model = ChatOpenAI(model="gpt-4o").bind(functions=functions)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [24]:
result = chain.invoke({"input": "search for blue hand bag"})

In [25]:
result

AgentActionMessageLog(tool='search_product', tool_input={'product_query': 'blue hand bag'}, log="\nInvoking: `search_product` with `{'product_query': 'blue hand bag'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"product_query":"blue hand bag"}', 'name': 'search_product'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 192, 'total_tokens': 210, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4691090a87', 'finish_reason': 'function_call', 'logprobs': None}, id='run-c27d24c7-8514-4500-9559-54bb865810ef-0', usage_metadata={'input_tokens': 192, 'output_tokens': 18, 'total_tokens': 210, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'r

In [27]:
result.tool

'search_product'

In [28]:
result.tool_input

{'product_query': 'blue hand bag'}

In [29]:
observation = search_product.invoke(result.tool_input)

In [30]:
observation

[{'asin': 'B0D5LJLM4L',
  'product_title': 'MakeMeChic Women&#x27;s Shoulder Bags Y2K PU Leather Clutch Small Purse Hobo Bags',
  'product_price': '$23.99',
  'product_original_price': None,
  'currency': 'USD',
  'product_star_rating': '4',
  'product_num_ratings': 1,
  'product_url': 'https://www.amazon.com/dp/B0D5LJLM4L',
  'product_photo': 'https://m.media-amazon.com/images/I/616Rp2RAibL._AC_UL960_FMwebp_QL65_.jpg',
  'product_num_offers': 1,
  'product_minimum_offer_price': '$23.99',
  'is_best_seller': False,
  'is_amazon_choice': False,
  'is_prime': True,
  'climate_pledge_friendly': False,
  'sales_volume': '50+ bought in past month',
  'delivery': 'FREE delivery on $35 shipped by Amazon.',
  'has_variations': True},
 {'asin': 'B06W5N7MG3',
  'product_title': 'Groupcow New Ladies Evening Handbags Bridal Wedding Bag Handbag',
  'product_price': '$11.99',
  'product_original_price': '$12.99',
  'currency': 'USD',
  'product_star_rating': '4.6',
  'product_num_ratings': 2009,
  '

# AgentFinish example with agent scratchpad

In [31]:
from langchain.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [32]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [34]:
result = chain.invoke({"input": "search for blue hand bag", "agent_scratchpad": []})

In [35]:
result

AgentActionMessageLog(tool='search_product', tool_input={'product_query': 'blue hand bag'}, log="\nInvoking: `search_product` with `{'product_query': 'blue hand bag'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"product_query":"blue hand bag"}', 'name': 'search_product'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 192, 'total_tokens': 210, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_50cad350e4', 'finish_reason': 'function_call', 'logprobs': None}, id='run-55f45045-d8f7-4c2b-8bfe-c5e292741539-0', usage_metadata={'input_tokens': 192, 'output_tokens': 18, 'total_tokens': 210, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'r

In [37]:
observation = search_product.invoke(result.tool_input)

In [38]:
from langchain.agents.format_scratchpad import format_to_openai_functions

In [39]:
format_to_openai_functions([(result, observation), ])

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"product_query":"blue hand bag"}', 'name': 'search_product'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 192, 'total_tokens': 210, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_50cad350e4', 'finish_reason': 'function_call', 'logprobs': None}, id='run-55f45045-d8f7-4c2b-8bfe-c5e292741539-0', usage_metadata={'input_tokens': 192, 'output_tokens': 18, 'total_tokens': 210, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 FunctionMessage(content='[{"asin": "B0D5LJLM4L", "product_title": "MakeMeChic Women&#x27;s Shoulder Bags Y2K PU Leather Clutch Small Purse Hobo Bags", "product_price": "$

In [40]:
result2 = chain.invoke({
    "input": "search for blue hand bag", 
    "agent_scratchpad": format_to_openai_functions([(result, observation)])
})

In [41]:
result2

AgentFinish(return_values={'output': "Here are some blue hand bags you might like:\n\n1. **MakeMeChic Women's Shoulder Bags Y2K PU Leather Clutch Small Purse Hobo Bags**\n   - Price: $23.99\n   - Rating: 4 stars from 1 rating\n   - Prime Delivery: Yes\n   - ![image](https://m.media-amazon.com/images/I/616Rp2RAibL._AC_UL960_FMwebp_QL65_.jpg)\n   - [View on Amazon](https://www.amazon.com/dp/B0D5LJLM4L)\n\n2. **Groupcow New Ladies Evening Handbags Bridal Wedding Bag Handbag**\n   - Price: $11.99 (Save 8% from $12.99)\n   - Rating: 4.6 stars from 2009 ratings\n   - Prime Delivery: Yes\n   - ![image](https://m.media-amazon.com/images/I/61-fO0yIsHL._AC_UL960_FMwebp_QL65_.jpg)\n   - [View on Amazon](https://www.amazon.com/dp/B06W5N7MG3)\n\n3. **Jorebill Shoulder Bags for Women, Vegan Leather Trendy Handbag Clutch Purse and Zipper Closure Tote Shoulder Purse**\n   - Price: $24.99\n   - Rating: 4.5 stars from 37 ratings\n   - Prime Delivery: Yes\n   - ![image](https://m.media-amazon.com/images/

In [42]:
print(result2.return_values["output"])

Here are some blue hand bags you might like:

1. **MakeMeChic Women's Shoulder Bags Y2K PU Leather Clutch Small Purse Hobo Bags**
   - Price: $23.99
   - Rating: 4 stars from 1 rating
   - Prime Delivery: Yes
   - ![image](https://m.media-amazon.com/images/I/616Rp2RAibL._AC_UL960_FMwebp_QL65_.jpg)
   - [View on Amazon](https://www.amazon.com/dp/B0D5LJLM4L)

2. **Groupcow New Ladies Evening Handbags Bridal Wedding Bag Handbag**
   - Price: $11.99 (Save 8% from $12.99)
   - Rating: 4.6 stars from 2009 ratings
   - Prime Delivery: Yes
   - ![image](https://m.media-amazon.com/images/I/61-fO0yIsHL._AC_UL960_FMwebp_QL65_.jpg)
   - [View on Amazon](https://www.amazon.com/dp/B06W5N7MG3)

3. **Jorebill Shoulder Bags for Women, Vegan Leather Trendy Handbag Clutch Purse and Zipper Closure Tote Shoulder Purse**
   - Price: $24.99
   - Rating: 4.5 stars from 37 ratings
   - Prime Delivery: Yes
   - ![image](https://m.media-amazon.com/images/I/71DKH2pyp+L._AC_UL960_FMwebp_QL65_.jpg)
   - [View on Am

# Agent loop using intermediate steps and agent scratchpad

In [43]:
known_actions = {
    "search_product": search_product,
    "add_to_cart": add_to_cart,
    "summarize_cart": summarize_cart,
    "buy_product": buy_product,  
}

In [44]:
from langchain.schema.agent import AgentFinish

def run_agent(user_input):
    intermediate_steps = []
    
    while True:
        result = chain.invoke({
            "input": user_input, 
            "agent_scratchpad": format_to_openai_functions(intermediate_steps)
        })
        
        if isinstance(result, AgentFinish):
            return result
            
        tool = known_actions[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))

In [45]:
from langchain.schema.runnable import RunnablePassthrough

agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | prompt | model | OpenAIFunctionsAgentOutputParser()

In [46]:
def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = agent_chain.invoke({
            "input": user_input, 
            "intermediate_steps": intermediate_steps
        })
        if isinstance(result, AgentFinish):
            return result
        tool = known_actions[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))

In [47]:
response = run_agent("search for blue hand bag")
print(response.return_values["output"])

Here are some blue handbags I found for you:

1. **MakeMeChic Women's Shoulder Bags**
   - Price: $23.99
   - Rating: ★★★★ (1 rating)
   - [View on Amazon](https://www.amazon.com/dp/B0D5LJLM4L)
   - ![Image](https://m.media-amazon.com/images/I/616Rp2RAibL._AC_UL960_FMwebp_QL65_.jpg)

2. **Groupcow New Ladies Evening Handbags**
   - Price: $11.99 (Original price: $12.99, Save 8%)
   - Rating: ★★★★☆ (2009 ratings)
   - [View on Amazon](https://www.amazon.com/dp/B06W5N7MG3)
   - ![Image](https://m.media-amazon.com/images/I/61-fO0yIsHL._AC_UL960_FMwebp_QL65_.jpg)

3. **Jorebill Shoulder Bags for Women**
   - Price: $24.99
   - Rating: ★★★★☆ (38 ratings)
   - [View on Amazon](https://www.amazon.com/dp/B0D4ZBGM41)
   - ![Image](https://m.media-amazon.com/images/I/71DKH2pyp+L._AC_UL960_FMwebp_QL65_.jpg)

Let me know if you want to add any of these to your cart, or if you need more information.


In [48]:
response

AgentFinish(return_values={'output': "Here are some blue handbags I found for you:\n\n1. **MakeMeChic Women's Shoulder Bags**\n   - Price: $23.99\n   - Rating: ★★★★ (1 rating)\n   - [View on Amazon](https://www.amazon.com/dp/B0D5LJLM4L)\n   - ![Image](https://m.media-amazon.com/images/I/616Rp2RAibL._AC_UL960_FMwebp_QL65_.jpg)\n\n2. **Groupcow New Ladies Evening Handbags**\n   - Price: $11.99 (Original price: $12.99, Save 8%)\n   - Rating: ★★★★☆ (2009 ratings)\n   - [View on Amazon](https://www.amazon.com/dp/B06W5N7MG3)\n   - ![Image](https://m.media-amazon.com/images/I/61-fO0yIsHL._AC_UL960_FMwebp_QL65_.jpg)\n\n3. **Jorebill Shoulder Bags for Women**\n   - Price: $24.99\n   - Rating: ★★★★☆ (38 ratings)\n   - [View on Amazon](https://www.amazon.com/dp/B0D4ZBGM41)\n   - ![Image](https://m.media-amazon.com/images/I/71DKH2pyp+L._AC_UL960_FMwebp_QL65_.jpg)\n\nLet me know if you want to add any of these to your cart, or if you need more information."}, log="Here are some blue handbags I foun

# Agent loop using AgentExecutor

In [49]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True)

In [50]:
response = agent_executor.invoke({"input": "search for blue hand bag"})

print(response["output"])



> Entering new AgentExecutor chain...

Invoking: `search_product` with `{'product_query': 'blue hand bag'}`


[{'asin': 'B0D5LJLM4L', 'product_title': 'MakeMeChic Women&#x27;s Shoulder Bags Y2K PU Leather Clutch Small Purse Hobo Bags', 'product_price': '$23.99', 'product_original_price': None, 'currency': 'USD', 'product_star_rating': '4', 'product_num_ratings': 1, 'product_url': 'https://www.amazon.com/dp/B0D5LJLM4L', 'product_photo': 'https://m.media-amazon.com/images/I/616Rp2RAibL._AC_UL960_FMwebp_QL65_.jpg', 'product_num_offers': 1, 'product_minimum_offer_price': '$23.99', 'is_best_seller': False, 'is_amazon_choice': False, 'is_prime': True, 'climate_pledge_friendly': False, 'sales_volume': '50+ bought in past month', 'delivery': 'FREE delivery on $35 shipped by Amazon.', 'has_variations': True}, {'asin': 'B06W5N7MG3', 'product_title': 'Groupcow New Ladies Evening Handbags Bridal Wedding Bag Handbag', 'product_price': '$11.99', 'product_original_price': '$12.99', 'currency': 'USD

In [51]:
agent_executor.invoke({"input": "add 1st product 2 quantity to the cart"})



> Entering new AgentExecutor chain...
I can't add anything to your cart without knowing which product you're talking about. Could you please provide a product query or specify which product you're referring to?

> Finished chain.


{'input': 'add 1st product 2 quantity to the cart',
 'output': "I can't add anything to your cart without knowing which product you're talking about. Could you please provide a product query or specify which product you're referring to?"}

# AgentExecutor with chat history

In [52]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [53]:
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | prompt | model | OpenAIFunctionsAgentOutputParser()

In [54]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

/var/folders/8x/zgklw70125n0ny5z111xstsr0000gn/T/ipykernel_87896/2202226372.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")


In [55]:
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True, memory=memory)

In [56]:
agent_executor.invoke({"input": "search for blue hand bag"})



> Entering new AgentExecutor chain...

Invoking: `search_product` with `{'product_query': 'blue hand bag'}`


[{'asin': 'B0D5LJLM4L', 'product_title': 'MakeMeChic Women&#x27;s Shoulder Bags Y2K PU Leather Clutch Small Purse Hobo Bags', 'product_price': '$23.99', 'product_original_price': None, 'currency': 'USD', 'product_star_rating': '4', 'product_num_ratings': 1, 'product_url': 'https://www.amazon.com/dp/B0D5LJLM4L', 'product_photo': 'https://m.media-amazon.com/images/I/616Rp2RAibL._AC_UL960_FMwebp_QL65_.jpg', 'product_num_offers': 1, 'product_minimum_offer_price': '$23.99', 'is_best_seller': False, 'is_amazon_choice': False, 'is_prime': True, 'climate_pledge_friendly': False, 'sales_volume': '50+ bought in past month', 'delivery': 'FREE delivery on $35 shipped by Amazon.', 'has_variations': True}, {'asin': 'B06W5N7MG3', 'product_title': 'Groupcow New Ladies Evening Handbags Bridal Wedding Bag Handbag', 'product_price': '$11.99', 'product_original_price': '$12.99', 'currency': 'USD

{'input': 'search for blue hand bag',
 'chat_history': [HumanMessage(content='search for blue hand bag', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Here are some fabulous blue handbags for you to check out:\n\n1. **MakeMeChic Women’s Shoulder Bags**\n   - Price: $23.99\n   - Rating: ⭐️⭐️⭐️⭐️ (1 rating)\n   - [View on Amazon](https://www.amazon.com/dp/B0D5LJLM4L)\n   - ![MakeMeChic Women's Shoulder Bags](https://m.media-amazon.com/images/I/616Rp2RAibL._AC_UL960_FMwebp_QL65_.jpg)\n\n2. **Groupcow New Ladies Evening Handbags**\n   - Price: $11.99 (was $12.99)\n   - Rating: ⭐️⭐️⭐️⭐️⭐️ (4.6 from 2009 ratings)\n   - [View on Amazon](https://www.amazon.com/dp/B06W5N7MG3)\n   - ![Groupcow New Ladies Evening Handbags](https://m.media-amazon.com/images/I/61-fO0yIsHL._AC_UL960_FMwebp_QL65_.jpg)\n\n3. **Jorebill Shoulder Bags for Women**\n   - Price: $24.99\n   - Rating: ⭐️⭐️⭐️⭐️⭐️ (4.5 from 38 ratings)\n   - [View on Amazon](https://www.amazon.com/dp/B0D4ZBGM41)\n   - ![Jor

In [57]:
agent_executor.invoke({"input": "add 1st product 2 quantity to the cart"})



> Entering new AgentExecutor chain...

Invoking: `add_to_cart` with `{'product_id': 'B0D5LJLM4L', 'quantity': 2}`


TrueI've added two of the fabulous **MakeMeChic Women’s Shoulder Bags** to your cart. If you need anything else or want to proceed to checkout, just let me know! 💼✨

> Finished chain.


{'input': 'add 1st product 2 quantity to the cart',
 'chat_history': [HumanMessage(content='search for blue hand bag', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Here are some fabulous blue handbags for you to check out:\n\n1. **MakeMeChic Women’s Shoulder Bags**\n   - Price: $23.99\n   - Rating: ⭐️⭐️⭐️⭐️ (1 rating)\n   - [View on Amazon](https://www.amazon.com/dp/B0D5LJLM4L)\n   - ![MakeMeChic Women's Shoulder Bags](https://m.media-amazon.com/images/I/616Rp2RAibL._AC_UL960_FMwebp_QL65_.jpg)\n\n2. **Groupcow New Ladies Evening Handbags**\n   - Price: $11.99 (was $12.99)\n   - Rating: ⭐️⭐️⭐️⭐️⭐️ (4.6 from 2009 ratings)\n   - [View on Amazon](https://www.amazon.com/dp/B06W5N7MG3)\n   - ![Groupcow New Ladies Evening Handbags](https://m.media-amazon.com/images/I/61-fO0yIsHL._AC_UL960_FMwebp_QL65_.jpg)\n\n3. **Jorebill Shoulder Bags for Women**\n   - Price: $24.99\n   - Rating: ⭐️⭐️⭐️⭐️⭐️ (4.5 from 38 ratings)\n   - [View on Amazon](https://www.amazon.com/dp/B0D4ZBGM4

In [58]:
cart

[{'product_id': 'B0D5LJLM4L', 'quantity': 2}]